In [1]:
import torch
import networkx as nx
import numpy as np
import pandas as pd
import time

# Fixed method parameters
n = 500
p = 0.5
gamma_c = 1
exploration_parameter_eta = 2.0
max_iters = 1500
tol = 1e-8
damping0 = 1e-3
ls_iters = 10
alpha0 = 900   # fixed learning rate
beta = 0.6    # fixed backtracking scale
N_TRIALS = 10  # number of random initializations per graph

# Functions
def objective(x, H):
    return -x.sum() + 0.5 * x @ (H @ x)

def gradient(x, H):
    return -torch.ones_like(x) + H @ x

def MIS_checker_efficient_3(X, A, A_bar):
    Xb = X.bool().float()
    Xb_up = Xb - 0.1 * (-torch.ones_like(Xb) + (n * A) @ Xb)
    Xb_up.clamp_(0,1)
    if torch.equal(Xb, Xb_up):
        return True, torch.nonzero(Xb).squeeze()
    return False, None

def projected_newton(H, A, A_bar, x0):
    x = x0.clone()
    damping = damping0
    I = torch.eye(n, dtype=H.dtype)
    H_damped = H + damping * I
    history = []
    is_MIS = False
    MIS = None

    for it in range(max_iters):
        g = gradient(x, H)
        g_norm = g.norm().item()
        f0 = objective(x, H).item()
        history.append((it, f0, g_norm))
        if g_norm < tol:
            break

        try:
            p = torch.linalg.solve(H_damped, g)
        except RuntimeError:
            damping *= 10
            H_damped = H + damping * I
            continue

        alpha = alpha0
        accepted = False
        for _ in range(ls_iters):
            x_trial = torch.clamp(x - alpha * p, 0.0, 1.0)
            if objective(x_trial, H).item() < f0:
                x = x_trial
                accepted = True
                break
            alpha *= beta

        if not accepted:
            damping *= 10

        damping = max(damping * 0.5, 1e-7)
        H_damped = H + damping * I

        is_MIS, MIS = MIS_checker_efficient_3(x, A, A_bar)
        if is_MIS:
            break

    return x, history, is_MIS, MIS

# Main batch testing over seeds 0-49
summaries = []
for seed in range(50):
    # Build graph for this seed
    g = nx.gnp_random_graph(n, p, seed=seed)
    A = torch.tensor(nx.adjacency_matrix(g).todense(), dtype=torch.float32)
    A_bar = torch.tensor(nx.adjacency_matrix(nx.complement(g)).todense(), dtype=torch.float32)

    # Build H
    degrees_c = dict(nx.complement(g).degree())
    gamma = 2 + max(degrees_c.values())
    H = gamma * A - gamma_c * A_bar

    # Degree init vector
    degrees = dict(g.degree())
    max_deg = max(degrees.values())
    d = torch.zeros(n)
    for node, deg in degrees.items():
        d[node] = 1 - deg / max_deg
    d_init = d / d.max()

    # Covariance
    cov = exploration_parameter_eta * torch.eye(n)

    # Run multiple trials
    trial_stats = []
    for init in range(N_TRIALS):
        if init == 0:
            x0 = d_init.clone()
        else:
            torch.manual_seed((seed << 16) + init)
            x0 = torch.normal(mean=d_init, std=torch.sqrt(torch.diag(cov)))
            x0 = torch.clamp(x0, 0.0, 1.0)

        start = time.time()
        x_opt, history, is_MIS, MIS = projected_newton(H, A, A_bar, x0)
        elapsed = time.time() - start
        trial_stats.append({
            'iters': len(history),
            'time': elapsed,
            'MIS_size': MIS.numel() if is_MIS else 0
        })

    df = pd.DataFrame(trial_stats)
    summaries.append({
        'seed': seed,
        'max_MIS_size': int(df.MIS_size.max()),
        'mean_iters': df.iters.mean(),
        'mean_time': df.time.mean()
    })

# Aggregate and display results
df_summary = pd.DataFrame(summaries)
print(f"Damped Projected Newton on ER({n},{p}) over seeds 0-49, 10 trials each, alpha={alpha0}, beta={beta}")
print(df_summary)

# Optionally save to CSV
df_summary.to_csv('newton_seed_summary.csv', index=False)

Damped Projected Newton on ER(500,0.5) over seeds 0-49, 10 trials each, alpha=900, beta=0.6
    seed  max_MIS_size  mean_iters  mean_time
0      0            10        82.2   0.174045
1      1            10        74.6   0.121747
2      2            10        74.3   0.110295
3      3            11        75.0   0.113356
4      4            12        64.5   0.095560
5      5            11        77.5   0.122571
6      6            11        73.6   0.122927
7      7            11        89.1   0.155784
8      8            11        77.5   0.125099
9      9            11        82.0   0.138810
10    10            11        72.8   0.118777
11    11            11        65.8   0.110839
12    12            11       100.4   0.160576
13    13            10        69.6   0.118556
14    14            12        69.1   0.110472
15    15            11        72.9   0.136280
16    16            11        71.6   0.128594
17    17            12        70.1   0.113513
18    18            11        68.7